In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables.utils import ConfigurableFieldSpec

In [4]:
promt = ChatPromptTemplate.from_messages(
    [
        ('system', '당신은 유능한 어시트턴트 입니다.'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{question}')
    ]
)

In [6]:
llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini'
)

In [7]:
output_parser = StrOutputParser()

In [8]:
chain = promt | llm | output_parser

In [9]:
def get_chat_history(user_id, conversation_id):
    chat_message_history = SQLChatMessageHistory(
        table_name=user_id, 
        session_id=conversation_id, 
        connection='sqlite:///sqlite.db'
    )

    return chat_message_history

In [11]:
config_fields = [
    ConfigurableFieldSpec(
        id="user_id",			
        annotation=str,
        name="User ID",			
        description="Unique identifier for a user.",
        default="",
        is_shared=True,
    ),
    
    ConfigurableFieldSpec(
        id="conversation_id",	
        annotation=str,
        name="Conversation ID",
        description="Unique identifier for a conversation.",
        default="",
        is_shared=True,
    ),
]

In [12]:
chain_with_history = RunnableWithMessageHistory(
    chain, 
    get_chat_history, 
    input_messages_key='question', 
    history_messages_key='chat_history', 
    history_factory_config=config_fields
)

In [13]:
config_dic = {
    'configurable': {'user_id': 'user1', 'conversation_id': 'conversation1'}
}

In [14]:
answer = chain_with_history.invoke(
    {'question':'안녕 반가워. 내 이름은 홍길동이야.'}, 
    config=config_dic
)

In [15]:
print(answer)

안녕하세요, 홍길동님! 반가워요. 어떻게 도와드릴까요?


In [16]:
answer = chain_with_history.invoke(
    {'question':'나는 코딩학원에서 프로그래밍을 교육하고 있어.'}, 
    config=config_dic
)

In [17]:
print(answer)

멋지네요! 프로그래밍 교육을 하신다니 정말 흥미로운 일입니다. 어떤 언어나 기술을 가르치고 계신가요? 학생들에게 어떤 내용을 주로 다루고 있는지도 궁금합니다!


In [18]:
answer = chain_with_history.invoke(
    {'question':'내 이름이 뭐라고?'}, 
    config=config_dic
)

In [19]:
print(answer)

홍길동님이라고 하셨습니다! 맞나요?
